In [1]:
from IPython.display import clear_output, display

import requests
from bs4 import BeautifulSoup
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', 400)
import numpy as np

import json
import time
import mouse
import os
from os.path import exists
import re


## Парсим структуру дерева категорий

In [2]:
def get_next_eapteka_level(prev_lvl: int, 
                           prev_res: pd.DataFrame, 
                           xpath_value: str='//*[@class="filter__links-list"]/li/a') -> pd.DataFrame:
    '''
    Рекурсия для парсинга категорий е-аптеки
    '''
    # Создаем балванку для аппенда результата
    lvl_res = pd.DataFrame()
    
    for i, lvl_prev in prev_res.iterrows():
        if not i%10:
            clear_output()
        
        # Исходное значение таблицы на текущем уровне
        lvl_tbl = pd.DataFrame({f'lvl{prev_lvl + 1}': [lvl_prev[-2]], 
                        f'lvl{prev_lvl + 1}_url': [lvl_prev[-1]]})
        
        if (len(lvl_prev) < 3) or (lvl_prev[-2] != lvl_prev[-3]):
            # Переходим на страницу
            driver.get(lvl_prev[f'lvl{prev_lvl}_url'])
            # Собираем категории и их ссылки
            lvl_categories = [el.text.capitalize() for el in driver.find_elements(by='xpath', value=xpath_value)]
            lvl_urls = [el.get_attribute('href') for el in driver.find_elements(by='xpath', value=xpath_value)]
            # Формируем датафрейм
            if lvl_categories:
                lvl_tbl = pd.DataFrame({f'lvl{prev_lvl + 1}': lvl_categories, 
                                        f'lvl{prev_lvl + 1}_url': lvl_urls})
            else:
                pass
        else:
            pass

        # Добавляем значения из предыдущей таблицы
        url_columns = [col for col in lvl_prev.index if not bool(re.match(r'.+_url', col))]
        [lvl_tbl.insert(0, url_col, lvl_prev[url_col]) for url_col in url_columns[::-1]]
        # Аппендим результат
        lvl_res = pd.concat([lvl_res, lvl_tbl]).reset_index(drop=True)
        print(prev_lvl + 1)
        print('Current result: ', lvl_res.shape)
        print()
    
    # Для проверки рузультатов
    globals()[f'lvl_res{prev_lvl + 1}'] = lvl_res.copy()
    
    # Фактор остановки углубления рекурсии
    if lvl_res.shape[0] == prev_res.shape[0]:
        return lvl_res
    else:
        return get_next_eapteka_level(prev_lvl=prev_lvl+1,
                                      prev_res=lvl_res)

In [3]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

In [ ]:
# Парсим категории
url = 'https://www.eapteka.ru/'
# Настраиваем нулевой уровень
lvl0_res = pd.DataFrame({'lvl0': ['Е-Аптека'], 'lvl0_url': [url]})

result = get_next_eapteka_level(prev_lvl=0,
                                prev_res=lvl0_res,
                                xpath_value='//*[@class="header__nav-item "]/a')
# Сохраняем категории
result.to_excel('./data/eapteka_categories.xlsx', index=False)

In [6]:
# Проверка результата
result = pd.read_excel('./data/eapteka_categories.xlsx')

for i in range(1, result.shape[1]):
    print(i, result.iloc[:, :i].drop_duplicates().shape)

1 (1, 1)
2 (9, 2)
3 (100, 3)
4 (557, 4)
5 (839, 5)
6 (842, 6)
7 (842, 7)


In [9]:
# Функции для парсинга айтемнеймов
def get_max_page_num(url: str) -> int:
    '''
    Возвращает номер последней страницы в категории
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        max_page = soup.find_all('li', attrs={'class': re.compile("ui-pagination.+number", re.I)})[-1].text.strip()
    except IndexError:
        max_page = 1
    
    return int(max_page)


def get_url_pagenation(url: str) -> list:
    '''
    Генератор списка страниц для посещения
    '''
    max_page = get_max_page_num(url)
    return [f'{url}?PAGEN_1={page_num}' for page_num in range(1, max_page + 1)]


def get_page_data(driver):
    '''
    Получаем данные со страницы
    '''
    item_names = [elem.text for elem in\
                  driver.find_elements(by='xpath', value='//h5[@class="listing-card__title"]')]
    return {'item_name': item_names}


In [10]:
# Подключаем драйвер
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(options=options)

In [ ]:
# Парсим айтем неймы  по категориям
file_name = './data/eapteka_item_names_1.csv'

# Подготавливаем списки категорий и страниц, которые уже есть в файле
if exists(file_name):
    data_table = pd.read_csv(file_name, sep='\t')
    data_table['body_url'] = data_table.apply(lambda x: x['lvl_url'].split('?')[0], axis=1)
    url_body_list = data_table['body_url'].unique().tolist()
    url_list = data_table[data_table['body_url'] == url_body_list[-1]]['lvl_url'].unique().tolist()
else:
    url_body_list = []
    url_list = []

for url in result.iloc[:, -1].tolist():
    # Проверка спасенности категории
    if url not in url_body_list[:-1]:
        for i, paged_url in enumerate(get_url_pagenation(url)):
            # Проверка спарсенности страницы
            if paged_url not in url_list:
            
                # Очистка вывода
                if not i % 5:
                    clear_output()
                print(paged_url)

                # Парсим текущую страницу
                driver.get(paged_url)

                # Получаем необходимые данные
                data_dict = get_page_data(driver)

                # Выводим данные
                print(list(zip(*data_dict.values()))[-1], end='\n\n')

                # Добавляем столбец с адресом
                parsed_data = pd.DataFrame(data_dict)
                parsed_data.insert(0, 'lvl_url', paged_url)

                # Сохраняем полученные данные
                parsed_data.to_csv(file_name, 
                                   mode='a' if exists(file_name) else 'w', 
                                   sep='\t', 
                                   index=False,
                                   header = not exists(file_name))
            else:
                continue
    else:
        continue
        
clear_output()
        

## Исследование данных

In [2]:
file_name = './data/eapteka_item_names.csv'

item_names = pd.read_csv(file_name, sep='\t').drop_duplicates()

In [4]:
item_names.to_excel('./data/eapteka_item_names.xlsx', index=False)

In [11]:
item_names['body_url'] = item_names.apply(lambda x: x['lvl_url'].split('?')[0], axis=1)
print(item_names.shape)

(107453, 3)


In [9]:
item_names

,lvl_url,item_name
0,https://www.eapteka.ru/goods/drugs/gynaecology...,"Гонадотропин хорионический, лиофилизат д/приг ..."
1,https://www.eapteka.ru/goods/drugs/gynaecology...,"ДляЖенс про, капсулы 100 мг 28 шт"
2,https://www.eapteka.ru/goods/drugs/gynaecology...,"ДляЖенс про, капсулы 200 мг 14 шт"
3,https://www.eapteka.ru/goods/drugs/gynaecology...,"Овитрель, раствор для п/к введ 250 мкг/0,5 мл ..."
4,https://www.eapteka.ru/goods/drugs/gynaecology...,"Оргалутран, раствор для п/к введ 0,25 мг/0,5 м..."
...,...,...
107813,https://www.eapteka.ru/goods/pribory_i_meditsi...,"Energizer Батарейки Lithium CR 2025 FSB1, 1 шт"
107814,https://www.eapteka.ru/goods/pribory_i_meditsi...,"Energizer Батарейки Max E92 AAA BP, 4 шт"
107815,https://www.eapteka.ru/goods/pribory_i_meditsi...,"Eveready Батарейки SHD C R14 FSB2, 2 шт"
107816,https://www.eapteka.ru/goods/pribory_i_meditsi...,"Energizer Батарейки Alkaline LR44 A76 FSB2, 2 шт"


In [45]:
full_tbl = result.merge(item_names, how='left', left_on='lvl6_url', right_on='body_url')\
                    .drop(columns=['body_url', 'lvl_url', 'lvl4', 'lvl5', 'lvl6', 'lvl6_url'])
full_tbl = full_tbl.drop_duplicates()

In [31]:
a = full_tbl.groupby('item_name', as_index=False).agg({'lvl0': 'count', 'lvl3': list})\
        .sort_values('lvl0', ascending=False).query('lvl0 > 1').reset_index(drop=True)

b = pd.DataFrame(a['lvl3'].apply(lambda x: '_'.join(x)))
b['lvl3'] = pd.DataFrame(a['lvl3'].apply(lambda x: '_'.join(x)))

b['a'] = 1
b.groupby('lvl3', as_index=False).count().sort_values('a', ascending=False).query('a > 1')

,lvl3,a
386,Для жирной кожи_Для зрелой кожи_Для комбинированной кожи_Для молодой кожи_Для сухой кожи_Для чувствительной кожи_Уходовая косметика,3063
374,Для жирной кожи_Для зрелой кожи_Для комбинированной кожи_Для молодой кожи_Для сухой кожи_Для чувствительной кожи,488
468,Для зрелой кожи_Уходовая косметика,400
440,Для жирной кожи_Уходовая косметика,280
515,Для сухой кожи_Уходовая косметика,275
424,Для жирной кожи_Для комбинированной кожи_Уходовая косметика,228
217,Бритвенные принадлежности_Товары для бритья,214
381,Для жирной кожи_Для зрелой кожи_Для комбинированной кожи_Для молодой кожи_Для сухой кожи_Для чувствительной кожи_Уход для век,206
529,Для чувствительной кожи_Уходовая косметика,200
506,Для сухой кожи_Для чувствительной кожи_Уходовая косметика,193


In [46]:
full_tbl['cnct1'] = full_tbl.apply(lambda x: f"{x['lvl1']}_{x['lvl2']}_{x['lvl3']}", axis=1)
full_tbl = full_tbl[['cnct1', 'item_name']]
joined = full_tbl.merge(full_tbl, how='inner', on='item_name').query('cnct1_x != cnct1_y')\
            .sort_values(['cnct1_x', 'cnct1_y'])



In [61]:
joined['full_count'] = joined.groupby('cnct1_x')['item_name'].transform('count')
joined['mix_count'] = joined.groupby(['cnct1_x', 'cnct1_y'])['item_name'].transform('count')
joined['metric'] = joined['mix_count'] / joined['full_count']
joined.drop(columns=['item_name', 'full_count', 'mix_count'])\
        .drop_duplicates()\
        .sort_values(['metric', 'cnct1_x', 'cnct1_y'], ascending=False).query('metric > 0.1')

,cnct1_x,cnct1_y,metric
277856,Медтовары_Средства ухода за лежачими больными_Подгузники для взрослых,Гигиена_Урологические прокладки_Урологические прокладки,1.000000
292680,"Медтовары_Солевые лампы, грелки_Грелки","Медтовары_Медицинские изделия и расходные материалы_Моче- и калоприемники, спринцовки",1.000000
292489,Медтовары_Презервативы и смазка_Презервативы,Медтовары_Медицинские изделия и расходные материалы_Материалы для узи и экг,1.000000
292774,Медтовары_Пластыри_Хирургические пластыри,Медтовары_Медицинские изделия и расходные материалы_Перевязочные материалы,1.000000
294769,Медтовары_Пластыри_Фиксирующие пластыри-катушки,Медтовары_Пластыри_Бактерицидные пластыри,1.000000
294774,Медтовары_Пластыри_Детские пластыри,Медтовары_Пластыри_Бактерицидные пластыри,1.000000
293677,Медтовары_Ортопедия_Пояса согревающие,Медтовары_Ортопедия_Бандажи,1.000000
294218,Медтовары_Ортопедия_Корсеты ортопедические,Медтовары_Ортопедия_Корректоры осанки,1.000000
294217,Медтовары_Ортопедия_Корректоры осанки,Медтовары_Ортопедия_Корсеты ортопедические,1.000000
293676,Медтовары_Ортопедия_Бандажи,Медтовары_Ортопедия_Пояса согревающие,1.000000
